In [59]:
import pandas as pd 
import numpy as np 
import scipy as sp 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt

# Text Processing

    Read each sentence from training file and remove the tabs and extract id and tweet seperately. 

In [60]:
file = "data/train_tweets.txt"
temp = []
with open(file, 'r') as data:
    for line in data:
        row = []
        line = line.replace('\t'," ")
        elem = line.strip().split(" ")
        row.append(elem[0])
        row.append(" ".join(elem[1:]))
        temp.append(row) 

In [61]:
tw = pd.DataFrame(temp,columns = ["User","Tweet"])
temp = []
elem = []

In [62]:
tw.head()

,User,Tweet
0,8746,@handle Let's try and catch up live next week!
1,8746,Going to watch Grey's on the big screen - Thur...
2,8746,@handle My pleasure Patrick....hope you are well!
3,8746,@handle Hi there! Been traveling a lot and lot...
4,8746,RT @handle Looking to Drink Clean & Go Green? ...


In [63]:
tw.tail()

,User,Tweet
328927,4357,Steelbox Demonstrates Open Video Framework wit...
328928,4357,Small Businesses Rely on Sage to Help Them Rid...
328929,4357,TimeSight Systems™ Announces Next-Generation P...
328930,4357,Diebold Makes Its Leading Monitoring Solutions...
328931,4357,GVI Security Solutions to Introduce AutoIP™ VM...


In [64]:
tw.shape

(328932, 2)

### Statistics: can be seen that number of tweets per user on the lower side

In [65]:
cnt_user = tw['User'].value_counts()
cnt_user.describe()

count    9297.000000
mean       35.380445
std        28.146449
min         1.000000
25%        18.000000
50%        32.000000
75%        38.000000
max       284.000000
Name: User, dtype: float64

In [66]:
X = tw.Tweet
y = tw.User
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [67]:
lr = LogisticRegression()
nb = MultinomialNB()
svm = LinearSVC()
def test_model(vect, model):
    X_train_dtm = vect.fit_transform(X_train)
    print('Features: ', X_train_dtm.shape[1])
    print(f'Rows: {X_train_dtm.shape[0]}')
    X_test_dtm = vect.transform(X_test)
    if model == 'LR':
        lr.fit(X_train_dtm, y_train)
        y_pred_class = lr.predict(X_test_dtm)
        algorithm = 'Logistic Regression'
    elif model == 'MNB':
        nb.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        algorithm = 'Multinomial Naive Bayes'
    elif model == 'SVM':
        svm.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        algorithm = 'Linear SVM'
    print('Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))
    print(algorithm)

In [69]:
vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 2)

In [70]:
test_model(vectorizer, 'MNB')

Features:  235290
Rows: 246699
Accuracy:  0.14320285043717243
Multinomial Naive Bayes


In [134]:
def prepare_test_data(vec):
    file1 = "data/test_tweets_unlabeled.txt"
    with open(file1, 'r') as data:
        temp = [line for line in data]    
    unlabel = pd.Series(temp)
    unlabel_dtm = vec.transform(unlabel)
    return unlabel_dtm
    
def submission_file(data):
    import csv
    with open('predicted.csv', 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerow(['Id','Predicted'])
        for count,predicted in enumerate(data):
            writer.writerow([count+1,predicted])

In [135]:
unlabel_dtm = prepare_test_data(vectorizer) 
unlabel_pred = nb.predict(unlabel_dtm)
submission_file(unlabel_pred)